In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
import json
import pandas as pd
import janitor

In [4]:
# list of names of the top 5 leagues
league_list = ['EPL','La_Liga','Bundesliga','Serie_A','Ligue_1']

# Loop through the website for each of the leagues, and scrape the player stats into a dataframe
league_dfs = {}
for league in league_list:
    URL = 'https://understat.com/league/' + league
    r = requests.get(URL)
    soup = BeautifulSoup(r.content,'html5lib')
    player_table = soup.findAll('script')[3]
    table_string = str(player_table).split("'")[1]
    json_string = table_string.encode('utf8').decode('unicode_escape')
    json_table = json.loads(json_string)
    league_df = pd.json_normalize(json_table)
    league_df = league_df.apply(pd.to_numeric, errors='ignore')
    league_df['League'] = league
    league_df['G90'] = league_df['goals'] / league_df['time'] * 90
    league_df['A90'] = league_df['assists'] / league_df['time'] * 90
    league_df['G+A90'] = league_df['G90'] + league_df['A90']
    league_df['xG90'] = league_df['xG'] / league_df['time'] * 90
    league_df['xA90'] = league_df['xA'] / league_df['time'] * 90
    league_df['xG+A90'] = league_df['xG90'] + league_df['xA90']
    league_dfs[league] = league_df
    
df = pd.concat([v for k,v in league_dfs.items()])

In [32]:
with open('player_data.json',encoding = 'utf-8') as json_data:
    json_data = json.load(json_data)

player_data = pd.json_normalize(json_data['PlayerData'])
player_data
# player_data = pd.read_json(json_data)
# player_data
# player_data = pd.DataFrame(data['songs'])
# df['lyrics']

,ID,Forename,Surname,ImageURL
0,9,Justin,HOYTE,https://cdn.soccerwiki.org/images/player/9.png
1,10,Gaël,CLICHY,https://cdn.soccerwiki.org/images/player/10.png
2,13,Johan,DJOUROU,https://cdn.soccerwiki.org/images/player/13.png
3,16,Sebastian,LARSSON,https://cdn.soccerwiki.org/images/player/16.png
4,19,Cesc,FÀBREGAS,https://cdn.soccerwiki.org/images/player/19.png
...,...,...,...,...
84975,118926,Víctor,BARRIOS,
84976,118927,Gustavo,VERÓN,
84977,118928,Bojan,KOVAČEVIĆ,
84978,118929,Ivan,VUKČEVIĆ,


In [ ]:
df.to_csv('player_data.csv', mode='w', index=False, header=True)